In [1]:
#!pip install crewai crewai_tools langchain_community sentence-transformers langchain-groq langchain_huggingface --quiet

In [19]:
#!pip install langchain

In [1]:
# set GROQ_API_KEY and TAVILY_API_KEY
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
from langchain.agents import tool
from crewai_tools import PDFSearchTool
from crewai import Agent, Crew, Task,Process
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI


/Users/tharhtet/.local/share/virtualenvs/Agentic_RAG-pWWUrD3e/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:913: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [3]:

"""
llm =ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    #model_name="llama3-8b-8192",
    model_name="llama-3.2-3b-preview",
    temperature=0.1,
    max_tokens=1000,

)
"""
# OR Use Ollama to run Locally
from langchain_community.llms import Ollama
llm = Ollama(
    model="llama3.2:latest",
    temperature=0,
    # other params...
)


In [4]:
#!pip install tf-keras

In [5]:

"""

rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

"""


rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="ollama", # or google, openai, anthropic, llama2, ...
            config=dict(
                #model="llama3-8b-8192",
                model="llama3.2:latest",
                base_url="http://localhost:11434"
            ),
        ),
        embedder=dict(
                provider="huggingface", # or openai, ollama, ...
                config=dict(
                    model="BAAI/bge-small-en-v1.5",
                   
                    # title="Embeddings",
                ),
            ),
    )
)



In [6]:
question = "How did self-attention mechanism evolve in large language models in pdf?"
rag_tool.run(question)

Using Tool: Search a PDF's content


'Relevant Content:\nThis makes it more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 22, 23, 19]. End-to-end memory networks are based on a recurrent attention mechanism instead of sequence- aligned recurrence and have been shown to perform well on simple-language question answering and language modeling tasks [28]. To the best of our knowledge, howev

In [7]:
@tool
def simple_router_tool(question : str) -> str:
  """Routes queries to the appropriate resources"""
  if "pdf" in question.lower():
    return "vectorstore"
  else:
    return "websearch"



In [8]:
test_input = {"question": "What does the  say about machine learning in pdf?"}
simple_router_tool.run(test_input)

'vectorstore'

In [ ]:
#Agent 1

Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

#Task 1
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    #tools=[simple_router_tool],
)

In [10]:
crew1 = Crew(
    agents = [Router_Agent],
    tasks = [router_task],
    verbose= True,
)
test_input = {"question": "What does the  say about machine learning in pdf?"}
result1 = crew1.kickoff(inputs=test_input)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question What does the  say about machine learning in pdf?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Action: simple_router_tool
Action Input: {"question": "What does the  say about machine learning in pdf?"} 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore




In [11]:
print(result1)

vectorstore


In [12]:
web_search_tool = TavilySearchResults()
question = "How did self-attention mechanism evolve in large language models in pdf?"
web_search_tool(question)

/Users/tharhtet/.local/share/virtualenvs/Agentic_RAG-pWWUrD3e/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'url': 'https://www.ewadirect.com/proceedings/ace/article/view/10056/pdf',
  'content': 'calculates similarity scores between this position and other positions, normalizes the scores using the Softmax function, and obtains attention weights α (i, j) associated with other positions. These weights are then applied to the corresponding Value vectors (typically feature vectors from the input sequence) to produce the final attention output. In large language models, the most common self-attention mechanisms are Scaled Dot-Product Attention and Multi-Head Attention. In Scaled Dot-Product [...] output 𝐴!for position i is obtained by concatenating or linearly combining the outputs from all attention heads: Ai = Concatenate <Ai 1, Ai 2, … , Ai h> (4) Proceedings of the 2023 International Conference on Machine Learning and Automation DOI: 10.54254/2755-2721/35/20230399 237 Presently, leading large language models, including GPT-3, BERT, RoBERTa, XLNet, Palm [9], and ChatGLM2 employ Multi-Head 

In [13]:
#Agent 2
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)




#Task 2
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
    tools=[rag_tool, web_search_tool],
)

In [14]:
# Crew for the System
simple_rag_crew = Crew(
    agents = [Router_Agent,Retriever_Agent],
    tasks = [router_task,retriever_task],
    verbose= True,
)


test_input = {"question" :  "How U-Net work?"}

#test_input = {"question" :  "Does dorthy have any friends?"}
result1 = simple_rag_crew.kickoff(inputs=test_input)

2025-05-18 10:39:44,480 - 8417615616 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How U-Net work?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Action: simple_router_tool
Action Input: {"question": "How U-Net work?"} 

websearch

Final Answer: websearch

> Finished chain.
 [DEBUG]: == [Router] Task output: websearch


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question How U-Net work? with the help of the respective tool.Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'.Use the rag_tool to retrieve information from the vectorstore in case the r

In [17]:
result1

'Agent stopped due to iteration limit or time limit.'

In [16]:
inputs ={"question":"How does self-attention mechanism help large language models in pdf?"}

#test_input = {"question" :  "Does dorthy have any friends?"}
result2 = simple_rag_crew.kickoff(inputs=inputs)
result2

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How does self-attention mechanism help large language models in pdf?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Action: simple_router_tool
Action Input: {"question": "How does self-attention mechanism help large language models in pdf?"} 

vectorstore

Thought: I should check if the question contains relevant keywords for vectorstore search.

Action: simple_router_tool
Action Input: {"question": "How does self-attention mechanism help large language models in pdf?"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I should check if the questi

'Agent stopped due to iteration limit or time limit.'